In [1]:
from nba_api.stats.endpoints import playercareerstats, playergamelog, playergamelogs
from nba_api.stats.static import players
import pandas as pd
from io import BytesIO

In [3]:
def get_player_id_from_name(player_name):
    all_players = players.get_players()
    for player in all_players:
        if player['full_name'].lower() == player_name.lower():
            return player['id']
    return None

In [11]:
# Set player name and retrieve player ID
player_name = "LeBron James"
player_id = get_player_id_from_name(player_name)

# Retrieve player career statistics
career = playercareerstats.PlayerCareerStats(player_id=player_id)
career_stats = career.get_data_frames()[0]
career_stats = career_stats.rename(columns={'TEAM_ABBREVIATION': 'TEAM'})

# Define column lists
totals_cols = ['MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'FGM', 'FGA',
            'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
            'OREB', 'DREB', 'PF']
per_game_cols = ['MPG', 'PPG', 'RPG', 'APG', 'SPG', 'BPG', 'TPG', 'FGM', 'FGA',
                'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
                'ORPG', 'DRPG', 'PF']
percentage_stats = ['FG_PCT', 'FG3_PCT', 'FT_PCT']

# Career Totals
career_totals_per_season = career_stats[['SEASON_ID', 'GP', 'TEAM'] + totals_cols].copy()
career_totals_per_season['FULL_NAME'] = player_name

# Sum career totals and convert to int
career_totals = {'FULL_NAME': player_name, 'GP': int(career_totals_per_season['GP'].sum())}
career_totals.update(career_totals_per_season[totals_cols].sum().astype(int).to_dict())

# Convert non-integer columns to int except for 'FULL_NAME', 'SEASON_ID', and percentage columns
for col in career_totals_per_season.columns:
    if col not in ['FULL_NAME', 'SEASON_ID', 'TEAM'] + percentage_stats:
        career_totals_per_season[col] = career_totals_per_season[col].astype(int)

# Career Averages
career_avg_per_season = pd.DataFrame(columns=['FULL_NAME', 'SEASON_ID', 'TEAM', 'GP'] + per_game_cols)

# Calculate per game averages
for stat, per_game_stat in zip(totals_cols, per_game_cols):
    career_avg_per_season[per_game_stat] = round(career_stats[stat] / career_stats['GP'], 1)

career_avg_per_season['SEASON_ID'] = career_stats['SEASON_ID']
career_avg_per_season['GP'] = career_stats['GP']
career_avg_per_season['FULL_NAME'] = player_name
career_avg_per_season['TEAM'] = career_stats['TEAM']

# Calculate career averages
career_averages = {'FULL_NAME': player_name, 'GP': int(career_avg_per_season['GP'].sum())}
career_averages.update(round(career_avg_per_season[per_game_cols].mean(), 1).to_dict())

# Convert percentage Stats
for stat in percentage_stats:
    career_avg_per_season[stat] = round(career_stats[stat] * 100, 1)
    career_averages[stat] = round(career_avg_per_season[stat].mean(), 1)
    career_totals_per_season[stat] = round(career_totals_per_season[stat] * 100, 1)
    career_totals[stat] = round(career_avg_per_season[stat].mean(), 1)

# Convert DataFrames to dictionaries
career_totals_per_season_json = career_totals_per_season.to_dict(orient='records')
career_avg_per_season_json = career_avg_per_season.to_dict(orient='records')

career_avg_per_season

,FULL_NAME,SEASON_ID,TEAM,GP,MPG,PPG,RPG,APG,SPG,BPG,...,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,ORPG,DRPG,PF
0,LeBron James,2003-04,CLE,79,39.5,20.9,5.5,5.9,1.6,0.7,...,41.7,0.8,2.7,29.0,4.4,5.8,75.4,1.3,4.2,1.9
1,LeBron James,2004-05,CLE,80,42.4,27.2,7.4,7.2,2.2,0.6,...,47.2,1.4,3.8,35.1,6.0,8.0,75.0,1.4,6.0,1.8
2,LeBron James,2005-06,CLE,79,42.5,31.4,7.0,6.6,1.6,0.8,...,48.0,1.6,4.8,33.5,7.6,10.3,73.8,0.9,6.1,2.3
3,LeBron James,2006-07,CLE,78,40.9,27.3,6.7,6.0,1.6,0.7,...,47.6,1.3,4.0,31.9,6.3,9.0,69.8,1.1,5.7,2.2
4,LeBron James,2007-08,CLE,75,40.4,30.0,7.9,7.2,1.8,1.1,...,48.4,1.5,4.8,31.5,7.3,10.3,71.2,1.8,6.1,2.2
5,LeBron James,2008-09,CLE,81,37.7,28.4,7.6,7.2,1.7,1.1,...,48.9,1.6,4.7,34.4,7.3,9.4,78.0,1.3,6.3,1.7
6,LeBron James,2009-10,CLE,76,39.0,29.7,7.3,8.6,1.6,1.0,...,50.3,1.7,5.1,33.3,7.8,10.2,76.7,0.9,6.4,1.6
7,LeBron James,2010-11,MIA,79,38.8,26.7,7.5,7.0,1.6,0.6,...,51.0,1.2,3.5,33.0,6.4,8.4,75.9,1.0,6.5,2.1
8,LeBron James,2011-12,MIA,62,37.5,27.1,7.9,6.2,1.9,0.8,...,53.1,0.9,2.4,36.2,6.2,8.1,77.1,1.5,6.4,1.5
9,LeBron James,2012-13,MIA,76,37.9,26.8,8.0,7.2,1.7,0.9,...,56.5,1.4,3.3,40.6,5.3,7.0,75.3,1.3,6.8,1.4


In [16]:
5.538095238095238/7.55238095238095

0.7332912988650695

In [56]:
def get_player_image(player_name):
    """
    Gets NBA player headshot using player ID.

    Args:
        - player_name (str): Full name of current or former NBA player.

    Returns:
        - URL link to the headshot of NBA player.
    """

    player_id = get_player_id_from_name(player_name=player_name)
    url = f'https://cdn.nba.com/headshots/nba/latest/1040x760/{player_id}.png'

    return url

get_player_image("Jason Kidd")

'https://cdn.nba.com/headshots/nba/latest/1040x760/467.png'